In [1]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [81]:
df = pd.read_csv("/home/rafael/git/projeto_bom_pagador/dataset/train.csv", index_col='ID')

In [82]:
df.rename(columns={'LIMIT_BAL': 'Renda Anual', 'SEX':'Sexo','EDUCATION':'Educacao','MARRIAGE':'Estado Civil', 'AGE':'Idade','PAY_0': 'Tempo Emp.',"default payment next month":"Situacao"}, inplace=True)

In [83]:
df = df[['Renda Anual', 'Sexo', 'Educacao', 'Estado Civil', 'Idade','Tempo Emp.', 'Situacao']]

In [84]:
idx = df[df['Educacao'] > 4].index
df['Educacao'].loc[idx] = 4

In [85]:
idx =df[(df['Estado Civil'] < 1) | (df['Estado Civil'] > 3)].index
df['Estado Civil'].loc[idx] = 3

In [86]:
idx = df[df['Tempo Emp.']<0].index
df['Tempo Emp.'].loc[idx] = 0

In [88]:
df

,Renda Anual,Sexo,Educacao,Estado Civil,Idade,Tempo Emp.,Situacao
ID,,,,,,,
1,20000,2,2,1,24,2,1
2,120000,2,2,2,26,0,1
3,90000,2,2,2,34,0,0
4,50000,2,2,1,37,0,0
5,50000,1,2,1,57,0,0
...,...,...,...,...,...,...,...
20996,50000,1,2,1,49,1,1
20997,230000,1,2,2,38,2,0
20998,80000,1,2,1,38,0,0


In [92]:
for x in df.index:
    df['Tempo Emp.'].loc[x] = random.randrange(1,10)

In [96]:
df['Situacao'].value_counts()

0    16204
1     4796
Name: Situacao, dtype: int64

In [141]:
# Balanceando a Base

ap_idx = df[df['Situacao']==0].iloc[0:9592].index
rep_idx = df[df['Situacao']==1].index

In [142]:
df2 = pd.concat([df.loc[ap_idx], df.loc[rep_idx]])
df2

,Renda Anual,Sexo,Educacao,Estado Civil,Idade,Tempo Emp.,Situacao
ID,,,,,,,
3,90000,2,2,2,34,8,0
4,50000,2,2,1,37,6,0
5,50000,1,2,1,57,5,0
6,50000,1,1,2,37,9,0
7,500000,1,1,2,29,6,0
...,...,...,...,...,...,...,...
20987,30000,1,2,1,45,1,1
20992,210000,1,3,1,38,6,1
20993,460000,1,1,2,44,4,1


Criando datasets para consultas internas e externas.

In [146]:
df2.to_csv("/home/rafael/git/projeto_bom_pagador/dataset/banco_original.csv")

consul_int, consul_ext = train_test_split(df2, test_size=0.5)

consul_int.to_csv("/home/rafael/git/projeto_bom_pagador/dataset/banco_interno.csv")
consul_ext.to_csv("/home/rafael/git/projeto_bom_pagador/dataset/banco_externo.csv")

Trabalhando com o modelo de classificação

In [143]:
x = df2.drop(['Situacao'], axis=1)
y = df2['Situacao']

In [144]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [145]:
clf = RandomForestClassifier(max_depth=15, random_state=0)
clf.fit(x_train, y_train)
resul = clf.predict(x_test)
accuracy_score(y_test, resul)*100

65.09149872596711